In [5]:
import json
import pandas as pd
import glob
with open('benchmarks/gitbug_java/samples_gitbugjava_instruct_.jsonl', 'r') as f:
    samples = [json.loads(line) for line in f]
samples_df = pd.DataFrame(samples)

# Iterate over rcl files
for rcl_file in glob.glob('results/gitbug_java/0_original/rcl_gitbugjava_patches_ir*_or*_bs10.json'):
    try:    
        with open(rcl_file, 'r') as f:
            data = json.load(f)
        data = [{"bug_id": k, "patches": v} for k, v in data.items()]
        # assert all(len(d["patches"]) == 10 for d in data)
        candidates_df = pd.DataFrame(data)
        # Get text before first </s>
        candidates_df["patches"] = candidates_df["patches"].apply(lambda x: [p.split("</s>")[0] for p in x])
        # Remove special tokens
        special_tokens = ["<s>", "</s>", "<EOT>", "<MID>", "<PRE>", "<SUF>"]
        candidates_df["patches"] = candidates_df["patches"].apply(lambda x: [p.replace(token, "") for token in special_tokens for p in x])
        # Rename patches to generation
        candidates_df = candidates_df.rename(columns={"patches": "generation"})
        merged_df = pd.merge(samples_df, candidates_df[["bug_id", "generation"]], left_on="identifier", right_on="bug_id", how="left").drop("bug_id", axis=1)
        merged_df.to_json(f'results/gitbug_java/0_original/candidates_gitbugjava_repairllama_{rcl_file.split("_")[-3]}x{rcl_file.split("_")[-2]}.jsonl', orient='records', lines=True)
    except Exception as e:
        print(f"Failed to process {rcl_file}: {e}")